# Chapter 1: Initial Query with ChatGPT

Dear user, you are part of a consulting firm tasked to: \
[1] identify opportunities for design improvements for a specific product \
[2] conduct benchmarking with competitors whenever appropriate.

In this initial stage, we will initialize the **product** \
and gather initial insights and directions via querying a LLM (**ChatGPT**).

### TO DO SECTION

In [1]:
'''
Dear user, enter your Product here!
'''

product = "PICO 4 All-in-One VR Headset"

### RUN AS INTENDED (DO NOT CHANGE ANYTHING.)

##### Set Up

In [2]:
! pip install python-dotenv

In [3]:
""" Set up OpenAI API key """
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")

In [4]:
""" Create directory """
import shutil

if os.path.exists('support'):                     # Delete 'support' folder if it exists
    shutil.rmtree('support')

search_terms = product

try:                                              # Create directory named after search terms
    os.makedirs("support/%s" % search_terms)
    print("Directory", search_terms, "created")
except FileExistsError:
    print("Directory", search_terms, "exists")

Directory PICO 4 All-in-One VR Headset created


In [5]:
try:                                              # Create directory to store ChatGPT
    os.makedirs("support/%s/%s" % (search_terms, "ChatGPT"))
    print("Directory ChatGPT created")
except FileExistsError:
    print("Directory ChatGPT exists")

Directory ChatGPT created


##### DESIGN OPPORTUNITIES QUERY with ChatGPT

In [6]:
""" Design Opportunities Query with ChatGPT """
import pickle
import pandas as pd
import json

from openai import OpenAI
client = OpenAI()

question =  f"Identify and rank ten opportunities for design improvements with the {product}."
template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised explanation with a maximum of 45 characters. If you don't know the answer to the question, strictly state 'I don't know'."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        # adjust persona and temperature
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"), "wb"))

data_string = pd.read_pickle("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)

try:
    components = list(data.keys())
    descriptions = list(data.values())

    df = pd.DataFrame({'Component': components, 'Description': descriptions})

    print("Design Opportunities Query with ChatGPT:")
    print(df)
except:
        print("Invalid Response. Please regenerate.")


Design Opportunities Query with ChatGPT:
          Component                           Description
0       Head Straps     Improve comfort and adjustability
1      Battery Life      Increase duration for longer use
2       Controllers  Enhance ergonomics and button layout
3     Audio Quality  Improve sound fidelity and immersion
4            Weight       Reduce for more comfortable use
5     Field of View         Increase for better immersion
6        Resolution          Increase for clearer visuals
7    User Interface              Simplify for ease of use
8  Heat Dissipation        Improve to prevent overheating
9   Motion Tracking   Enhance accuracy and responsiveness


##### BENCHMARKING QUERY with ChatGPT

In [9]:
""" BENCHMARKING QUERY with ChatGPT """

""" Get 3 competitors """
import pickle
import pandas as pd
import json

search_terms = product

from openai import OpenAI
client = OpenAI()

question =  f"Identify three competitors of the {product} and the top ten physical components of each competitor."
template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as the product name (do not add numbering) and the value as a Python list of the ten components. If you don't know the answer to the question, strictly state ' '."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        # adjust persona and temperature
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"), "wb"))

""" Benchmarking """
print("Benchmarking Query with ChatGPT:")
data_string = pd.read_pickle("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)
try:
    competitors = list(data.keys())
    components = list(data.values())
    client = OpenAI()

    for i in range(len(competitors)):
        question = f"Using the {product}, conduct benchmarking with {competitors[i]} with regards to these ten physical components: {data[competitors[i]]}."
        template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised explanation with a maximum of 45 characters. If you don't know the answer to the question, strictly state 'I don't know.'."
        prompt = question + template

        chat_completion = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,  # adjust persona and temperature
        )

        result = chat_completion.choices[0].message.content

        pickle.dump(result, open("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]), "wb"))

        data_string = pd.read_pickle("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]))
        benchmarking_data = json.loads(data_string)
        try:
            components = list(benchmarking_data.keys())
            descriptions = list(benchmarking_data.values())

            df = pd.DataFrame({'Component': components, 'Benchmarking': descriptions})

            print(competitors[i])
            print(df)
            print("\n")
        except:
            print("Invalid Response. Please regenerate.")
except:
        print("Invalid Response. Please regenerate.")

Benchmarking Query with ChatGPT:
Oculus Quest 2
            Component                                  Benchmarking
0             Headset  PICO lighter & smaller, Oculus more powerful
1   Touch Controllers         Oculus has more intuitive controllers
2       Power Adapter           Both provide similar power adapters
3      Glasses Spacer         Only Oculus provides a glasses spacer
4      Charging Cable          Both have similar length and quality
5          Head Strap  Oculus strap more comfortable and adjustable
6        AA Batteries      Oculus controllers use AA, PICO built-in
7         USB-C Cable          Both headsets use USB-C for charging
8          Lens Cloth      Both provide lens cloth, similar quality
9  Instruction Manual             Both have clear, detailed manuals


HTC Vive Cosmos
           Component                                    Benchmarking
0            Headset       Pico more lightweight, HTC has higher res
1        Controllers                   Pico uses